In [1]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import numpy as np
import os
load_dotenv()

e:\1. Daily Activity\1_Artifical_Intelligence\1_Artificial_Intelligence\1_AI_Zafar_Iqbal\3_Projects_Sir Zaffar\3_Hospital_Management_System\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# 1. Data Ingestion

In [4]:
# ## load all the text files from the directory
# dir_loader=DirectoryLoader(
#     "data",
#     glob="**/*.pdf", ## Pattern to match files  
#     loader_cls= PyMuPDFLoader, ##loader class to use
#     show_progress=True

# )

# pdf_documents=dir_loader.load()
# print(f"Total Pages Loaded: {len(pdf_documents)}")

In [5]:
# type(pdf_documents[0])

In [2]:
def load_medical_documents(directory_path: str):
    """
    Loads all PDF documents from the specified directory using PyMuPDFLoader.
    
    Args:
        directory_path (str): The path to the folder containing PDFs.
        
    Returns:
        list: A list of loaded LangChain Document objects.
    """
    if not os.path.exists(directory_path):
        raise FileNotFoundError(f"The directory '{directory_path}' does not exist.")

    print(f"Starting to load documents from: {directory_path}...")

    # Initialize the DirectoryLoader with PyMuPDF for high speed and accuracy
    dir_loader = DirectoryLoader(
        directory_path,
        glob="**/*.pdf",
        loader_cls=PyMuPDFLoader,
        show_progress=True
    )

    try:
        documents = dir_loader.load()
        print(f"Successfully loaded {len(documents)} pages.")
        return documents
    except Exception as e:
        print(f"An error occurred while loading: {e}")
        return []

pdf_documents = load_medical_documents("data")

Starting to load documents from: data...


100%|██████████| 1/1 [00:17<00:00, 17.14s/it]

Successfully loaded 4505 pages.


# 2. Data Parsing

In [7]:
# #Split the Data into Text Chunks
# def text_split(extracted_data):
#     text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#     text_chunks=text_splitter.split_documents(extracted_data)
#     return text_chunks

In [8]:
# text_chunks=text_split(pdf_documents)
# print("Length of Text Chunks", len(text_chunks))

In [5]:
def split_medical_documents(documents, chunk_size=1000, chunk_overlap=200):
    """
    Splits loaded PDF documents into smaller, manageable chunks.
    Optimal for high-density medical text.
    """
    print("Initializing Text Splitter...")
    
    # Using RecursiveCharacterTextSplitter to keep medical context together
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ".", " ", ""] # Added "." to prefer splitting at sentences
    )
    
    try:
        split_docs = text_splitter.split_documents(documents)
        print(f"✅ Success: Split {len(documents)} pages into {len(split_docs)} chunks.")
        
        # Displaying a sample for verification
        if split_docs:
            print("-" * 30)
            print(f"Sample Chunk Metadata: {split_docs[0].metadata}")
            print(f"Sample Chunk Preview: {split_docs[0].page_content[:150]}...")
            print("-" * 30)
            
        return split_docs
        
    except Exception as e:
        print(f"❌ Error during splitting: {e}")
        return []
    
chunks = split_medical_documents(pdf_documents)

Initializing Text Splitter...
✅ Success: Split 4505 pages into 23586 chunks.
------------------------------
Sample Chunk Metadata: {'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'creator': 'Adobe Acrobat 6.0', 'creationdate': '2006-10-16T20:19:33+02:00', 'source': 'data\\The Gale Encyclopedia of Medicine Vol 1-5.pdf', 'file_path': 'data\\The Gale Encyclopedia of Medicine Vol 1-5.pdf', 'total_pages': 4505, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2006-10-16T22:03:45+02:00', 'trapped': '', 'modDate': "D:20061016220345+02'00'", 'creationDate': "D:20061016201933+02'00'", 'page': 1}
Sample Chunk Preview: The GALE
ENCYCLOPEDIA of
MEDICINE
THIRD EDITION...
------------------------------


# 3. Embedding

In [10]:
# #Download the Embeddings from Hugging Face
# def download_hugging_face_embeddings():
#     embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#     return embeddings

In [11]:
# embeddings = download_hugging_face_embeddings()

In [12]:
# query_result = embeddings.embed_query("medical Chatbot")
# print("Length", len(query_result))

In [6]:
class MedicalEmbeddingManager:
    """Handles medical document embeddings using LangChain's HuggingFace integration"""
    
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager.
        Standardizes the model for Pinecone compatibility (Dimension: 384).
        """
        self.model_name = model_name
        self.embeddings = self._load_model()

    def _load_model(self):
        """Load the HuggingFace embedding model via LangChain"""
        try:
            print(f"Loading embedding model: {self.model_name}...")
            # This returns a LangChain compatible embedding object
            model = HuggingFaceEmbeddings(
                model_name=self.model_name,
                model_kwargs={'device': 'cpu'}, # Use 'cuda' if you have a GPU
                encode_kwargs={'normalize_embeddings': True} # Better for Cosine Similarity
            )
            print("✅ Model loaded successfully.")
            return model
        except Exception as e:
            print(f"❌ Error loading model {self.model_name}: {e}")
            raise

    def get_embeddings_object(self):
        """Returns the embedding object for direct use in Vector Stores"""
        return self.embeddings

# Initialize the manager
embedding_manager = MedicalEmbeddingManager()
medical_embeddings = embedding_manager.get_embeddings_object()

Loading embedding model: sentence-transformers/all-MiniLM-L6-v2...
✅ Model loaded successfully.


# 4. Vector Store

In [4]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
PINECONE_INDEX_NAME=os.environ.get('PINECONE_INDEX_NAME')

In [15]:
# pc = Pinecone(api_key=PINECONE_API_KEY)
# index_name = "medical-index"

# # 2. Check karein ke index pehle se toh nahi bana hua
# if index_name not in pc.list_indexes().names():
#     print(f"Creating index: {index_name}")
#     pc.create_index(
#         name=index_name,
#         dimension=384, # all-MiniLM-L6-v2 ke liye fix hai
#         metric='cosine', # Similarity check karne ke liye best hai
#         spec=ServerlessSpec(
#             cloud='aws', 
#             region='us-east-1'
#         )
#     )
#     print("✅ Index successfully created!")
# else:
#     print("ℹ️ Index already exists.")


In [16]:
# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name=PINECONE_INDEX_NAME,
#     embedding=embeddings, 
# )

In [17]:
# Load Existing index 

# Embed each chunk and upsert the embeddings into your Pinecone index.
# docsearch = PineconeVectorStore.from_existing_index(
#     index_name=PINECONE_INDEX_NAME,
#     embedding=embeddings
# )

In [18]:
# retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [19]:
# retriever.invoke("What is Diabetes?")

In [20]:
# class MedicalVectorStore:
#     """Manages medical document embeddings in a Pinecone vector store"""
    
#     def __init__(self, index_name: str = "medical-chatbot", dimension: int = 384):
#         """
#         Initialize Pinecone connection and index.
#         Args:
#             index_name: Name of the Pinecone index.
#             dimension: Dimension for all-MiniLM-L6-v2 (default 384).
#         """
#         self.index_name = index_name
#         self.api_key = os.getenv("PINECONE_API_KEY")
#         self.pc = Pinecone(api_key=self.api_key)
#         self._initialize_index(dimension)

#     def _initialize_index(self, dimension):
#         """Create index if it doesn't exist"""
#         try:
#             if self.index_name not in self.pc.list_indexes().names():
#                 print(f"Creating new Pinecone index: {self.index_name}...")
#                 self.pc.create_index(
#                     name=self.index_name,
#                     dimension=dimension,
#                     metric='cosine',
#                     spec=ServerlessSpec(cloud='aws', region='us-east-1')
#                 )
#                 print("✅ Index created successfully.")
#             else:
#                 print(f"✅ Using existing index: {self.index_name}")
#         except Exception as e:
#             print(f"❌ Error initializing Pinecone: {e}")
#             raise

#     def add_documents(self, chunks, embedding_model):
#         """
#         Uploads chunks to Pinecone and returns the vectorstore object.
#         Args:
#             chunks: List of split LangChain documents.
#             embedding_model: The embedding object from MedicalEmbeddingManager.
#         """
#         try:
#             print(f"Uploading {len(chunks)} chunks to Pinecone. This may take time...")
            
#             # Using LangChain's optimized wrapper
#             vectorstore = PineconeVectorStore.from_documents(
#                 documents=chunks,
#                 embedding=embedding_model,
#                 index_name=self.index_name
#             )
            
#             print(f"✅ Successfully indexed all documents in '{self.index_name}'")
#             return vectorstore
            
#         except Exception as e:
#             print(f"❌ Error adding documents to Pinecone: {e}")
#             raise

# # --- INITIALIZATION ---
# # 1. Initialize the manager
# vector_manager = MedicalVectorStore(index_name=PINECONE_INDEX_NAME)

# # 2. Add documents (Using chunks from splitter and embeddings from manager)
# vectorstore = vector_manager.add_documents(chunks, medical_embeddings)

In [ ]:
# Pinecone mein duplicate entries se bachne ke liye unique IDs generate karna zaroori hai. Yeh IDs content ke hash par based hongi taake agar content same hai to ID bhi same ra

import hashlib

class MedicalVectorStore:
    """Manages medical document embeddings in a Pinecone vector store"""
    
    def __init__(self, index_name: str = "medical-chatbot", dimension: int = 384):
        """
        Initialize Pinecone connection and index.
        Args:
            index_name: Name of the Pinecone index.
            dimension: Dimension for all-MiniLM-L6-v2 (default 384).
        """
        self.index_name = index_name
        self.api_key = os.getenv("PINECONE_API_KEY")
        self.pc = Pinecone(api_key=self.api_key)
        self._initialize_index(dimension)

    def _initialize_index(self, dimension):
        """Create index if it doesn't exist"""
        try:
            if self.index_name not in self.pc.list_indexes().names():
                print(f"Creating new Pinecone index: {self.index_name}...")
                self.pc.create_index(
                    name=self.index_name,
                    dimension=dimension,
                    metric='cosine',
                    spec=ServerlessSpec(cloud='aws', region='us-east-1')
                )
                print("✅ Index created successfully.")
            else:
                print(f"✅ Using existing index: {self.index_name}")
        except Exception as e:
            print(f"❌ Error initializing Pinecone: {e}")
            raise

    def add_documents(self, chunks, embedding_model):
        try:
            print(f"Checking/Uploading {len(chunks)} chunks to Pinecone...")
            
            # 1. Initialize VectorStore object
            vectorstore = PineconeVectorStore(
                index_name=self.index_name, 
                embedding=embedding_model
            )

            # 2. Generate Unique IDs based on content hash
            # Is se agar text same hai to ID same rahegi (No Duplicates)
            ids = []
            for chunk in chunks:
                # Text aur page number ko mila kar hash banayein
                header_info = f"{chunk.metadata.get('source', '')}_{chunk.metadata.get('page', '')}"
                content_hash = hashlib.md5(chunk.page_content.encode()).hexdigest()
                unique_id = f"{header_info}_{content_hash}"
                ids.append(unique_id)

            # 3. Upsert documents (IDs ke sath)
            # Pinecone check karega: Agar ID maujud hai to Update, warna Insert.
            vectorstore.add_documents(documents=chunks, ids=ids, batch_size=100)
            
            print(f"✅ Successfully processed documents in '{self.index_name}'")
            return vectorstore
            
        except Exception as e:
            print(f"❌ Error: {e}")
            raise

# 1. Initialize the manager
vector_manager = MedicalVectorStore(index_name=PINECONE_INDEX_NAME)

# 2. Add documents (Using chunks from splitter and embeddings from manager)
vectorstore = vector_manager.add_documents(chunks, medical_embeddings)

In [7]:
vectorstore = PineconeVectorStore(
    index_name=PINECONE_INDEX_NAME, 
    embedding=medical_embeddings
)
print("✅ Connected to existing data in Pinecone. No need to wait for upload!")

✅ Connected to existing data in Pinecone. No need to wait for upload!


# 5. Retriever Pipeline From VectorStore

In [8]:
from typing import List, Dict, Any

class MedicalRAGRetriever:
    """Handles query-based retrieval from the Pinecone vector store"""
    
    def __init__(self, vectorstore):
        """
        Initialize the retriever.
        Args:
            vectorstore: The LangChain PineconeVectorStore instance we created earlier.
        """
        self.vectorstore = vectorstore

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.5) -> List[Dict[str, Any]]:
        """
        Retrieve relevant medical documents for a query.
        """
        print(f"🔍 Searching for: '{query}'...")
        
        try:
            # LangChain automatically handles embedding generation for the query
            # and searches in Pinecone.
            results = self.vectorstore.similarity_search_with_score(
                query, 
                k=top_k
            )
            
            retrieved_docs = []
            
            for doc, score in results:
                # Score threshold filter (Optional but recommended for quality)
                # Note: Pinecone scores are often Cosine Similarity (0 to 1)
                if score >= score_threshold:
                    retrieved_docs.append({
                        'content': doc.page_content,
                        'metadata': doc.metadata,
                        'similarity_score': score,
                        'source': doc.metadata.get('source', 'Unknown'),
                        'page': doc.metadata.get('page', 'Unknown')
                    })
            
            print(f"✅ Found {len(retrieved_docs)} relevant medical contexts.")
            return retrieved_docs
            
        except Exception as e:
            print(f"❌ Error during retrieval: {e}")
            return []

# Initialize the retriever with the vectorstore we got from the previous step
rag_retriever = MedicalRAGRetriever(vectorstore)

In [9]:
# Test a query
results = rag_retriever.retrieve("What are the symptoms of Diabetes?", top_k=3)

🔍 Searching for: 'What are the symptoms of Diabetes?'...
✅ Found 3 relevant medical contexts.


In [10]:
rag_retriever.retrieve("What is Diabetes?")

🔍 Searching for: 'What is Diabetes?'...
✅ Found 5 relevant medical contexts.


[{'content': 'Diabetes mellitus\nDefinition\nDiabetes mellitus is a condition in which the pan-\ncreas no longer produces enough insulin or cells stop\nresponding to the insulin that is produced, so that\nglucose in the blood cannot be absorbed into the\ncells of the body. Symptoms include frequent urina-\ntion, lethargy, excessive thirst, and hunger. The treat-\nment includes changes in diet, oral medications, and in\nsome cases, daily injections of insulin.\nDescription\nDiabetes mellitus is a chronic disease that causes\nserious health complications including renal (kidney)\nfailure,\nheart\ndisease,\nstroke,\nand\nblindness.\nApproximately 17 million Americans have diabetes.\nUnfortunately, as many as one-half are unaware they\nhave it.\nBackground\nEvery cell in the human body needs energy in\norder to function. The body’s primary energy source\nis glucose, a simple sugar resulting from the digestion\nof\nfoods\ncontaining\ncarbohydrates\n(sugars\nand\nstarches). Glucose from the 

# 6. Context + Prompt (VectorDB To LLM Output Generation)

In [12]:
GOOGle_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [33]:
class MedicalGeminiLLM:
    def __init__(self, model_name: str = "gemini-flash-latest"):
        self.api_key = os.environ.get("GOOGLE_API_KEY")
        
        # Increased max_output_tokens to 2048 to prevent incomplete answers
        self.llm = ChatGoogleGenerativeAI(
            model=model_name,
            google_api_key=self.api_key,
            temperature=0.1,
            max_output_tokens=2048, 
            transport="rest",
            safety_settings={
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
            }
        )
        print(f"✅ Professional Gemini System Online: {model_name}")

    def generate_medical_response(self, query: str, retrieved_docs: list) -> str:
        # Extract meaningful source names and pages from metadata
        context_parts = []
        for i, doc in enumerate(retrieved_docs):
            source_file = doc['metadata'].get('source', 'Medical Reference')
            page_num = doc['metadata'].get('page', 'N/A')
            # Cleaning path to show only filename
            source_name = os.path.basename(source_file)
            
            context_parts.append(f"REFERENCE [{source_name}, Page {page_num}]:\n{doc['content']}")
        
        context_text = "\n\n".join(context_parts)
        
        # Professional Medical Prompt
        template = """
        You are a highly qualified Medical Consultant. Provide a detailed and structured answer 
        using the provided medical references. 

        RULES:
        1. Base your answer STRICTLY on the provided references.
        2. At the end of each paragraph, cite the specific [File Name, Page] used.
        3. Use professional medical terminology and structure (e.g., Symptoms, Etiology, Risk Factors).
        4. Provide a full, comprehensive response. Do not cut off mid-sentence.
        5. If the reference doesn't contain the answer, state that "Current documentation does not provide details on this".

        MEDICAL REFERENCES:
        {context}

        PATIENT QUESTION: 
        {question}

        CONSULTANT ADVICE:
        """
        
        prompt = PromptTemplate.from_template(template)
        formatted_prompt = prompt.format(context=context_text, question=query)
        
        try:
            response = self.llm.invoke([HumanMessage(content=formatted_prompt)])
            return response.content
        except Exception as e:
            return f"❌ System Error: {str(e)}"

# Initialize
gemini_llm = MedicalGeminiLLM()

Unexpected argument 'transport' provided to ChatGoogleGenerativeAI.
C:\Users\aiengineer\AppData\Local\Temp\ipykernel_12880\4169794993.py:62: UserWarning: WARNING! transport is not default parameter.
                transport was transferred to model_kwargs.
                Please confirm that transport is what you intended.
  gemini_llm = MedicalGeminiLLM()


✅ Professional Gemini System Online: gemini-flash-latest


In [34]:
class MedicalChatbot:
    """
    Final Workflow Class that connects the Retriever and the LLM.
    """
    def __init__(self, vectorstore, gemini_manager):
        self.retriever = MedicalRAGRetriever(vectorstore)
        self.llm_manager = gemini_manager

    def ask(self, question: str, top_k: int = 4):
        print(f"\n{'='*100}")
        print(f"USER QUESTION: {question}")
        print(f"{'='*100}\n")

        # 1. RETRIEVAL STEP: Find relevant chunks from your 23,586 records
        retrieved_docs = self.retriever.retrieve(question, top_k=top_k)

        if not retrieved_docs:
            return "I couldn't find any relevant information in the medical database."

        # 2. GENERATION STEP: Send context to Gemini for a professional answer
        final_answer = self.llm_manager.generate_medical_response(question, retrieved_docs)
        
        return final_answer

# --- Final Chatbot Setup ---
# Step 1: Setup the Chatbot System
# 'vectorstore' is the object you got after uploading your 23,586 chunks
chatbot = MedicalChatbot(vectorstore, gemini_llm)

# Step 2: Start Chatting!
user_query = "What are the primary symptoms and risk factors of Hypertension?"
response = chatbot.ask(user_query)

print("\n--- AI MEDICAL ASSISTANT RESPONSE ---\n")
print(response)


USER QUESTION: What are the primary symptoms and risk factors of Hypertension?

🔍 Searching for: 'What are the primary symptoms and risk factors of Hypertension?'...
✅ Found 4 relevant medical contexts.

--- AI MEDICAL ASSISTANT RESPONSE ---

**CONSULTANT ADVICE:**

### Clinical Presentation and Symptoms of Hypertension

Hypertension is characterized as a major health problem, primarily because it is largely asymptomatic. Many individuals have hypertension without being aware of their condition. A diagnosis of hypertension is not based on temporary increases in blood pressure—such as those caused by physical activity or stressful situations—but is established only when a person has multiple high blood pressure readings recorded over a period of time. [The Gale Encyclopedia of Medicine Vol 1-5.pdf, Page 1930.0; The Gale Encyclopedia of Medicine Vol 1-5.pdf, Page 1931.0]

### Etiology and Risk Factors

The cause of hypertension remains unknown in 90 to 95 percent of affected individuals

In [35]:
user_query = "What is blood sugar level? and its normal range?"
response = chatbot.ask(user_query)

print("\n--- AI MEDICAL ASSISTANT RESPONSE ---\n")
print(response)


USER QUESTION: What is blood sugar level? and its normal range?

🔍 Searching for: 'What is blood sugar level? and its normal range?'...
✅ Found 4 relevant medical contexts.

--- AI MEDICAL ASSISTANT RESPONSE ---

As a highly qualified Medical Consultant, I will provide a detailed and structured overview of blood glucose levels and their associated normal and abnormal ranges, based strictly on the provided medical references.

### Blood Glucose Level Measurement

Current documentation does not provide a specific definition for "blood sugar level" itself, but references detail the measurement and interpretation of blood glucose levels used for diagnostic purposes, such as screening for insulin resistance or diabetes. Blood glucose tests are utilized to determine if the body is not producing sufficient insulin or is not using it effectively. Two common tests used for screening are the fasting glucose test and the glucose tolerance test. [The Gale Encyclopedia of Medicine Vol 1-5.pdf, Pag

In [39]:
user_query = "what is blood cancer and its types?"
response = chatbot.ask(user_query)

print("\n--- AI MEDICAL ASSISTANT RESPONSE ---\n")
print(response)


USER QUESTION: what is blood cancer and its types?

🔍 Searching for: 'what is blood cancer and its types?'...
✅ Found 4 relevant medical contexts.

--- AI MEDICAL ASSISTANT RESPONSE ---

As a highly qualified Medical Consultant, I will provide a detailed overview of blood cancer and its types based strictly on the provided medical documentation.

### Definition of Blood Cancer

Cancers that originate in the blood and lymph glands are specifically categorized as leukemias and lymphomas, respectively. Leukemia is defined as a disease in which the cells that constitute the blood become cancerous or abnormal. Cells that possess the ability to invade are described as malignant. [The Gale Encyclopedia of Medicine Vol 1-5.pdf, Page 731.0; The Gale Encyclopedia of Medicine Vol 1-5.pdf, Page 1700.0]

### Types of Blood Cancer

The documentation identifies three primary types of cancer affecting blood cells and related tissues: Leukemia, Lymphoma, and Multiple Myeloma.

#### 1. Leukemia
Leukemi

In [40]:
user_query = "what is diabetes and its types?"
response = chatbot.ask(user_query)

print("\n--- AI MEDICAL ASSISTANT RESPONSE ---\n")
print(response)


USER QUESTION: what is diabetes and its types?

🔍 Searching for: 'what is diabetes and its types?'...
✅ Found 4 relevant medical contexts.

--- AI MEDICAL ASSISTANT RESPONSE ---

Diabetes mellitus is a chronic condition defined by the body’s inability to properly regulate glucose, the primary energy source derived from the digestion of carbohydrates. This occurs either because the pancreas no longer produces sufficient insulin, or because the body’s cells cease responding effectively to the insulin that is produced, thereby preventing glucose in the blood from being absorbed into the cells. Diabetes mellitus is a chronic disease that can lead to serious health complications, including renal failure, heart disease, stroke, and blindness. [The Gale Encyclopedia of Medicine Vol 1-5.pdf, Page 1184.0]

### Clinical Manifestations and Treatment

The symptoms associated with diabetes mellitus include lethargy, excessive thirst, and hunger. A key symptom is frequent urination, which is medica

In [41]:
user_query = "What is sugar and its effects on human body?"
response = chatbot.ask(user_query)

print("\n--- AI MEDICAL ASSISTANT RESPONSE ---\n")
print(response)


USER QUESTION: What is sugar and its effects on human body?

🔍 Searching for: 'What is sugar and its effects on human body?'...
✅ Found 4 relevant medical contexts.

--- AI MEDICAL ASSISTANT RESPONSE ---

The following consultation details the definition, metabolic role, regulation, and pathological effects of sugar (glucose and carbohydrates) on the human body, based strictly on the provided medical references.

### I. Definition and Classification of Sugar

Carbohydrates, which include both sugars and starches, constitute one of the three major nutrients in the human diet and serve as the primary source of energy [The Gale Encyclopedia of Medicine Vol 1-5.pdf, Page 750.0]. Carbohydrates are classified based on their structure, specifically the number of basic sugar, or saccharide units, they contain [The Gale Encyclopedia of Medicine Vol 1-5.pdf, Page 750.0]. A monosaccharide represents the simplest carbohydrate structure and is referred to as a simple sugar [The Gale Encyclopedia o

In [42]:
user_query = "What is AI"
response = chatbot.ask(user_query)

print("\n--- AI MEDICAL ASSISTANT RESPONSE ---\n")
print(response)


USER QUESTION: What is AI

🔍 Searching for: 'What is AI'...
✅ Found 0 relevant medical contexts.

--- AI MEDICAL ASSISTANT RESPONSE ---

I couldn't find any relevant information in the medical database.
